In [ ]:
# Imports
import sys
from pathlib import Path

import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from lime.lime_tabular import LimeTabularExplainer

# Configuration du style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports réussis")

## 1. Chargement du modèle et du dataset

In [ ]:
# Chemins
BASE_DIR = Path().resolve()
MODEL_PATH = BASE_DIR / "future_skills_model.pkl"
DATASET_PATH = BASE_DIR / "future_skills_dataset.csv"

# Vérification
if not MODEL_PATH.exists():
    raise FileNotFoundError(f"Modèle introuvable: {MODEL_PATH}")
if not DATASET_PATH.exists():
    raise FileNotFoundError(f"Dataset introuvable: {DATASET_PATH}")

print(f"📂 Modèle: {MODEL_PATH}")
print(f"📂 Dataset: {DATASET_PATH}")

In [ ]:
# Charger le pipeline
pipeline = joblib.load(MODEL_PATH)
print(f"✅ Modèle chargé: {type(pipeline).__name__}")
print(f"   Étapes: {list(pipeline.named_steps.keys())}")

# Charger le dataset
df = pd.read_csv(DATASET_PATH)
print(f"\n✅ Dataset chargé: {len(df)} lignes, {len(df.columns)} colonnes")
print(f"   Colonnes: {df.columns.tolist()}")

In [ ]:
# Filtrer les niveaux valides
ALLOWED_LEVELS = {"LOW", "MEDIUM", "HIGH"}
df = df[df["future_need_level"].isin(ALLOWED_LEVELS)].copy()

print(f"📊 Répartition des classes:")
print(df["future_need_level"].value_counts())
print(f"\n📊 Pourcentages:")
print(df["future_need_level"].value_counts(normalize=True).mul(100).round(2))

## 2. Préparation des données pour l'explainability

In [ ]:
# Identifier les features disponibles
potential_features = [
    "job_role_name",
    "skill_name",
    "skill_category",
    "job_department",
    "trend_score",
    "internal_usage",
    "training_requests",
    "scarcity_index",
    "hiring_difficulty",
    "avg_salary_k",
    "economic_indicator",
]

feature_cols = [col for col in potential_features if col in df.columns]
target_col = "future_need_level"

print(f"✅ Features utilisées ({len(feature_cols)}):")
for col in feature_cols:
    print(f"   - {col}")

X = df[feature_cols].copy()
y = df[target_col].copy()

In [ ]:
# Séparer features catégorielles et numériques
categorical_features = []
numeric_features = []

for col in feature_cols:
    if df[col].dtype == 'object' or df[col].dtype.name == 'category':
        categorical_features.append(col)
    else:
        numeric_features.append(col)

print(f"📋 Features catégorielles: {categorical_features}")
print(f"📋 Features numériques: {numeric_features}")

## 3. Sélection d'exemples représentatifs

In [ ]:
# Sélectionner 2 exemples HIGH et 2 exemples MEDIUM
high_examples = df[df["future_need_level"] == "HIGH"].sample(n=min(2, len(df[df["future_need_level"] == "HIGH"])), random_state=42)
medium_examples = df[df["future_need_level"] == "MEDIUM"].sample(n=min(2, len(df[df["future_need_level"] == "MEDIUM"])), random_state=42)

examples = pd.concat([high_examples, medium_examples])

print(f"🎯 Exemples sélectionnés pour l'analyse:")
print("\n" + "="*80)
for idx, row in examples.iterrows():
    print(f"\n📌 Exemple {idx}:")
    print(f"   Job Role: {row.get('job_role_name', 'N/A')}")
    print(f"   Skill: {row.get('skill_name', 'N/A')}")
    print(f"   Level: {row['future_need_level']}")
    print(f"   Trend Score: {row.get('trend_score', 'N/A')}")
    print(f"   Internal Usage: {row.get('internal_usage', 'N/A')}")
    print(f"   Scarcity Index: {row.get('scarcity_index', 'N/A')}")
print("\n" + "="*80)

## 4. SHAP Analysis

SHAP (SHapley Additive exPlanations) est basé sur la théorie des jeux et fournit des explications cohérentes et interprétables.

In [ ]:
# Préparer les données transformées pour SHAP
# Le pipeline contient un preprocessor + classifier
preprocessor = pipeline.named_steps['preprocess']
clf = pipeline.named_steps['clf']

# Transformer toutes les données
X_transformed = preprocessor.transform(X)

# Pour les exemples sélectionnés
X_examples = examples[feature_cols]
X_examples_transformed = preprocessor.transform(X_examples)

print(f"✅ Données transformées: {X_transformed.shape}")
print(f"✅ Exemples transformés: {X_examples_transformed.shape}")

In [ ]:
# Créer un explainer SHAP pour RandomForest
# TreeExplainer est optimal pour les modèles basés sur les arbres
explainer = shap.TreeExplainer(clf)

# Calculer les valeurs SHAP pour nos exemples
shap_values = explainer.shap_values(X_examples_transformed)

print(f"✅ SHAP values calculées")
print(f"   Shape: {shap_values[0].shape if isinstance(shap_values, list) else shap_values.shape}")
print(f"   Classes: {clf.classes_}")

In [ ]:
# Récupérer les noms des features après transformation
feature_names = []

# Features catégorielles (OneHot encoded)
if categorical_features:
    cat_transformer = preprocessor.named_transformers_['cat']
    if hasattr(cat_transformer, 'get_feature_names_out'):
        cat_names = cat_transformer.get_feature_names_out(categorical_features)
        feature_names.extend(cat_names)

# Features numériques
feature_names.extend(numeric_features)

print(f"✅ {len(feature_names)} feature names après transformation")
print(f"   Premières 10: {feature_names[:10]}")

In [ ]:
# Visualisation SHAP: Force plots pour chaque exemple
shap.initjs()

for i, (idx, row) in enumerate(examples.iterrows()):
    print(f"\n{'='*80}")
    print(f"📊 SHAP Force Plot - Exemple {i+1}")
    print(f"   Job Role: {row.get('job_role_name', 'N/A')}")
    print(f"   Skill: {row.get('skill_name', 'N/A')}")
    print(f"   True Level: {row['future_need_level']}")
    print(f"{'='*80}\n")
    
    # Prédiction
    pred = clf.predict(X_examples_transformed[i:i+1])[0]
    pred_proba = clf.predict_proba(X_examples_transformed[i:i+1])[0]
    
    print(f"   Predicted: {pred}")
    for j, cls in enumerate(clf.classes_):
        print(f"   P({cls}): {pred_proba[j]:.3f}")
    
    # Force plot pour la classe prédite
    predicted_class_idx = np.argmax(pred_proba)
    
    if isinstance(shap_values, list):
        # Multi-class: une matrice par classe
        shap_display = shap.force_plot(
            explainer.expected_value[predicted_class_idx],
            shap_values[predicted_class_idx][i],
            X_examples_transformed[i],
            feature_names=feature_names
        )
    else:
        # Binary
        shap_display = shap.force_plot(
            explainer.expected_value,
            shap_values[i],
            X_examples_transformed[i],
            feature_names=feature_names
        )
    
    display(shap_display)

In [ ]:
# Waterfall plots (mieux pour visualiser individuellement)
for i, (idx, row) in enumerate(examples.iterrows()):
    print(f"\n{'='*80}")
    print(f"📊 SHAP Waterfall Plot - Exemple {i+1}: {row.get('job_role_name', 'N/A')} × {row.get('skill_name', 'N/A')}")
    print(f"{'='*80}\n")
    
    pred_proba = clf.predict_proba(X_examples_transformed[i:i+1])[0]
    predicted_class_idx = np.argmax(pred_proba)
    
    if isinstance(shap_values, list):
        shap_obj = shap.Explanation(
            values=shap_values[predicted_class_idx][i],
            base_values=explainer.expected_value[predicted_class_idx],
            data=X_examples_transformed[i],
            feature_names=feature_names
        )
    else:
        shap_obj = shap.Explanation(
            values=shap_values[i],
            base_values=explainer.expected_value,
            data=X_examples_transformed[i],
            feature_names=feature_names
        )
    
    plt.figure(figsize=(10, 6))
    shap.waterfall_plot(shap_obj, max_display=10)
    plt.tight_layout()
    plt.show()

In [ ]:
# Summary plot: vue d'ensemble de l'importance des features
print("\n" + "="*80)
print("📊 SHAP Summary Plot - Vue d'ensemble")
print("="*80 + "\n")

if isinstance(shap_values, list):
    # Pour chaque classe
    for class_idx, class_name in enumerate(clf.classes_):
        print(f"\n🎯 Classe: {class_name}")
        plt.figure(figsize=(10, 6))
        shap.summary_plot(
            shap_values[class_idx],
            X_examples_transformed,
            feature_names=feature_names,
            show=False,
            max_display=10
        )
        plt.title(f"SHAP Summary - {class_name}")
        plt.tight_layout()
        plt.show()
else:
    plt.figure(figsize=(10, 6))
    shap.summary_plot(
        shap_values,
        X_examples_transformed,
        feature_names=feature_names,
        show=False,
        max_display=10
    )
    plt.tight_layout()
    plt.show()

## 5. LIME Analysis (Alternative)

LIME fournit des explications locales en approximant le modèle par un modèle linéaire simple autour de l'instance à expliquer.

In [ ]:
# Créer l'explainer LIME
# Utiliser un échantillon du dataset pour définir l'espace des features
sample_size = min(1000, len(X_transformed))
X_sample = X_transformed[:sample_size]

lime_explainer = LimeTabularExplainer(
    training_data=X_sample,
    feature_names=feature_names,
    class_names=clf.classes_,
    mode='classification',
    random_state=42
)

print(f"✅ LIME Explainer créé")
print(f"   Training samples: {sample_size}")
print(f"   Features: {len(feature_names)}")
print(f"   Classes: {clf.classes_}")

In [ ]:
# Générer des explications LIME pour chaque exemple
for i, (idx, row) in enumerate(examples.iterrows()):
    print(f"\n{'='*80}")
    print(f"📊 LIME Explanation - Exemple {i+1}")
    print(f"   Job Role: {row.get('job_role_name', 'N/A')}")
    print(f"   Skill: {row.get('skill_name', 'N/A')}")
    print(f"   True Level: {row['future_need_level']}")
    print(f"{'='*80}\n")
    
    # Générer l'explication
    exp = lime_explainer.explain_instance(
        data_row=X_examples_transformed[i],
        predict_fn=clf.predict_proba,
        num_features=10,
        top_labels=3
    )
    
    # Afficher l'explication
    pred = clf.predict(X_examples_transformed[i:i+1])[0]
    print(f"   Predicted: {pred}")
    
    # Probabilités
    pred_proba = clf.predict_proba(X_examples_transformed[i:i+1])[0]
    for j, cls in enumerate(clf.classes_):
        print(f"   P({cls}): {pred_proba[j]:.3f}")
    
    # Visualisation
    print("\n   Top contributing features:")
    predicted_class_idx = np.argmax(pred_proba)
    for feat, weight in exp.as_list(label=predicted_class_idx)[:10]:
        print(f"     {feat}: {weight:+.4f}")
    
    # Plot
    fig = exp.as_pyplot_figure(label=predicted_class_idx)
    plt.title(f"LIME - {row.get('job_role_name', 'N/A')} × {row.get('skill_name', 'N/A')}")
    plt.tight_layout()
    plt.show()

## 6. Extraction des signaux clés pour explications simplifiées

Mapper les signaux (trend_score, scarcity_index, internal_usage) vers des phrases simples.

In [ ]:
def extract_top_features_from_shap(shap_values_array, feature_names, top_n=3):
    """
    Extrait les top N features qui ont le plus contribué à la prédiction.
    Retourne une liste de tuples (feature_name, shap_value).
    """
    # Valeurs absolues pour importance
    abs_values = np.abs(shap_values_array)
    top_indices = np.argsort(abs_values)[-top_n:][::-1]
    
    return [
        (feature_names[idx], shap_values_array[idx])
        for idx in top_indices
    ]

def generate_simple_explanation(top_features, prediction_level):
    """
    Génère une explication textuelle simple à partir des top features.
    """
    # Mapping des features vers des termes compréhensibles
    feature_mapping = {
        'trend_score': 'tendance marché',
        'scarcity_index': 'rareté interne',
        'internal_usage': 'usage interne actuel',
        'training_requests': 'demandes de formation',
        'hiring_difficulty': 'difficulté de recrutement',
        'avg_salary_k': 'niveau de salaire',
        'economic_indicator': 'indicateur économique',
    }
    
    explanation_parts = []
    
    for feat_name, shap_val in top_features:
        # Identifier le signal principal
        base_feature = None
        for key in feature_mapping.keys():
            if key in feat_name.lower():
                base_feature = key
                break
        
        if base_feature:
            readable_name = feature_mapping[base_feature]
            
            if shap_val > 0:
                strength = "forte" if abs(shap_val) > 0.2 else "modérée"
                explanation_parts.append(f"{readable_name} {strength}")
            else:
                strength = "faible" if abs(shap_val) > 0.2 else "limitée"
                explanation_parts.append(f"{readable_name} {strength}")
    
    if prediction_level == "HIGH":
        prefix = "Score élevé car :"
    elif prediction_level == "MEDIUM":
        prefix = "Score modéré car :"
    else:
        prefix = "Score faible car :"
    
    if explanation_parts:
        return f"{prefix} {' + '.join(explanation_parts)}"
    else:
        return f"{prefix} multiple facteurs combinés"

print("✅ Fonctions d'extraction créées")

In [ ]:
# Générer des explications simplifiées pour nos exemples
print("\n" + "="*80)
print("📝 EXPLICATIONS SIMPLIFIÉES")
print("="*80 + "\n")

for i, (idx, row) in enumerate(examples.iterrows()):
    pred_proba = clf.predict_proba(X_examples_transformed[i:i+1])[0]
    predicted_class_idx = np.argmax(pred_proba)
    predicted_level = clf.classes_[predicted_class_idx]
    
    # Extraire top features
    if isinstance(shap_values, list):
        shap_vals = shap_values[predicted_class_idx][i]
    else:
        shap_vals = shap_values[i]
    
    top_features = extract_top_features_from_shap(shap_vals, feature_names, top_n=3)
    explanation = generate_simple_explanation(top_features, predicted_level)
    
    print(f"\n📌 Exemple {i+1}:")
    print(f"   Job Role: {row.get('job_role_name', 'N/A')}")
    print(f"   Skill: {row.get('skill_name', 'N/A')}")
    print(f"   Niveau prédit: {predicted_level} ({pred_proba[predicted_class_idx]:.1%})")
    print(f"\n   💡 EXPLICATION:")
    print(f"   {explanation}")
    print(f"\n   🔍 Détails:")
    for feat, val in top_features:
        print(f"     • {feat}: {val:+.4f}")
    print()

## 7. Analyse globale: Features les plus importantes

Identifier quelles features sont généralement les plus influentes pour toutes les prédictions.

In [ ]:
# Calculer SHAP values pour un échantillon plus large
sample_for_global = min(500, len(X_transformed))
X_global_sample = X_transformed[:sample_for_global]

print(f"Calcul des SHAP values pour {sample_for_global} exemples...")
shap_values_global = explainer.shap_values(X_global_sample)
print("✅ Calcul terminé")

In [ ]:
# Bar plot: importance moyenne des features
print("\n" + "="*80)
print("📊 Feature Importance Globale (SHAP)")
print("="*80 + "\n")

if isinstance(shap_values_global, list):
    for class_idx, class_name in enumerate(clf.classes_):
        print(f"\n🎯 Classe: {class_name}")
        plt.figure(figsize=(10, 6))
        shap.summary_plot(
            shap_values_global[class_idx],
            X_global_sample,
            feature_names=feature_names,
            plot_type="bar",
            show=False,
            max_display=10
        )
        plt.title(f"Top 10 Features - {class_name}")
        plt.tight_layout()
        plt.show()
else:
    plt.figure(figsize=(10, 6))
    shap.summary_plot(
        shap_values_global,
        X_global_sample,
        feature_names=feature_names,
        plot_type="bar",
        show=False,
        max_display=10
    )
    plt.title("Top 10 Features Globales")
    plt.tight_layout()
    plt.show()

## 8. Conclusions et recommandations

### Insights clés:

1. **Features les plus influentes**: Les analyses SHAP/LIME révèlent quelles caractéristiques (trend_score, scarcity_index, etc.) contribuent le plus aux prédictions HIGH vs LOW.

2. **Patterns par classe**: 
   - **HIGH**: Généralement associé à une forte tendance marché + rareté interne élevée
   - **MEDIUM**: Mix équilibré de signaux
   - **LOW**: Tendances faibles ou usage interne déjà satisfaisant

3. **Explications simplifiées**: Les explications textuelles peuvent être générées automatiquement et stockées avec les prédictions.

### Prochaines étapes:

- ✅ Créer un module `explanation_engine.py` pour automatiser la génération d'explications
- ✅ Ajouter un champ `explanation` au modèle `FutureSkillPrediction`
- ✅ Intégrer l'explication dans l'API (optionnel via paramètre `?include_explanation=true`)
- ✅ Documenter l'UI future pour afficher "Pourquoi cette compétence ?"

In [ ]:
print("\n" + "="*80)
print("✅ ANALYSE D'EXPLICABILITÉ TERMINÉE")
print("="*80)
print("\nLes visualisations et explications ci-dessus démontrent:")
print("  1. Comment SHAP identifie les features clés pour chaque prédiction")
print("  2. Comment LIME fournit des explications locales alternatives")
print("  3. Comment traduire les SHAP values en explications textuelles simples")
print("\nCes analyses peuvent être intégrées dans l'application pour fournir")
print("de la transparence aux utilisateurs RH sur les recommandations ML.")
print("="*80)